# Scrape The Most up-to-date Monthly Insolvency Statistics

The purpose of these functions is to
- Find and read the most recent monthly insolvency statistics file
- Read each of the sheets in the file into a dataframe
- Get a long and short description of the data
- Collect the unique column values from the dataframes
- Find the date of the most recent data available

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [ ]:
#| export
def get_insolvency_stats():
    """
    Find the most up to date monthly insolvency statistics and fetch the 
    url so it can be used to collect the .xlsx file.

    Open the webpage and read the HTML.
    Find the .xlsx file by searching through the <a> elements.
    (we know it's in an <a> element as this is how links are defined in HTML)

    Returns a dictionary where the keys are the sheet names and the values are
    dataframes containing the sheet data.
    """
    base_url = "https://www.gov.uk"
    url = base_url + "/government/collections/monthly-insolvency-statistics"

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        uls = soup.find("ul", {"class": "gem-c-document-list"})

        insolvency_stats_url = uls.findChildren("a")[0]['href'] # link for first item in list (most recent stats)
        insolvency_stats_url = base_url + insolvency_stats_url

        response = requests.get(insolvency_stats_url)

        # Opening the relevant file
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            for a in soup.find_all("a", {"class": "govuk-link"}):
                if 'xlsx' in a['href']:
                    r = requests.get(a['href'])
                    file = pd.ExcelFile(r.content)

        # Reading the methods and notes from sheets in the document to create a long description
        for sheet in file.sheet_names:
            if 'Method' in sheet:
                MaQ = file.parse(sheet)[1:]
            if 'Note' in sheet:
                notes = file.parse(sheet).iloc[1:,:2]

        MaQ = '\n'.join(MaQ.iloc[:, 0])
        notes.iloc[:, 0] = '[' + notes.iloc[:, 0] + ']'
        notes = notes.iloc[: ,0] + ' ' + notes.iloc[:, 1]
        notes = '\n'.join(notes)
        long_description = MaQ + ' \n' + notes

        # Extracting the sheets that contain the actual data
        dfs = {}

        for sheet in file.sheet_names:
            if 'Table' in sheet:
                title = file.parse(sheet).keys()[0]
                dfs[title] = file.parse(sheet) # save as df in dictionary

        for table in dfs:
            skip_rows = np.argmax(~dfs[table].iloc[:, 2].isnull()) # find the number of redundant rows
            dfs[table].columns = dfs[table].iloc[skip_rows, :]  # Replace columns with correct values
            dfs[table] = dfs[table][skip_rows+1:]   # ignore the empty rows
        
    return dfs, long_description

In [ ]:
insolvency_stats, long_desc = get_insolvency_stats()
print('Title: ' + list(insolvency_stats.keys())[0])
print('Description: ' + long_desc[29:276])

Title: Table 1: Registered Company insolvencies, England and Wales, 1 January 2019 to 31 May 2023, not seasonally adjusted [p][note 1][note 2]
Description: 
The Insolvency Service is currently releasing monthly company and individual insolvency statistics for England & Wales and Northern Ireland, and monthly company statistics for Scotland, to provide more up to date information on insolvency volumes


In [ ]:
list(insolvency_stats.values())[0].head(3)

4,Month,Total company insolvencies,Total company insolvencies revised?,Compulsory liquidations [note 3],Compulsory liquidations revised?,Creditors' voluntary liquidations [note 4],Creditors' voluntary liquidations revised?,Administrations,Administrations revised?,Company voluntary arrangements,Company voluntary arrangements revised?,Receivership appointments,Receivership appointments revised?
5,Jan 19,1726,NaN,317,NaN,1171,NaN,196,NaN,42,NaN,0,NaN
6,Feb 19,1213,NaN,259,NaN,806,NaN,125,NaN,23,NaN,0,NaN
7,Mar 19,1581,NaN,238,NaN,1139,NaN,175,NaN,29,NaN,0,NaN


In [ ]:
#| export
def get_insolvency_unique_column_vals(data):
    "Given a dataframe, this function finds the unique column values"
    col_data = {}
    data = data.reset_index(drop=True)

    for col in data.columns:

        if type(data.loc[:, col][0]) == str: # Check for string data type
            if not data.loc[:, col][0].replace('.','', 1).isdigit(): # if the data is a string ensure that it isn't numeric
                col_data[col] = list(data.loc[:, col].unique())

    return col_data

In [ ]:
#| export
def get_mis_last_updated():
    "Gets the publish date of the most up to date insolvency data"
    base_url = "https://www.gov.uk"
    url = base_url + "/government/collections/monthly-insolvency-statistics"

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        last_updated_element = soup.find(text="Last updated")
        last_updated = last_updated_element.find_next()
        last_updated = last_updated.text.strip().split('\n')[0]
    else:
        last_updated = float('NaN')

    return last_updated

In [ ]:
get_mis_last_updated()

'16 June 2023'

In [ ]:
#| export
def get_mis_description():
    "Gets a short description of the insolvency statistics dataset"
    base_url = "https://www.gov.uk"
    url = base_url + "/government/collections/monthly-insolvency-statistics"

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the <p> element with a class containing the word "lead"
        lead_paragraph = soup.select("p[class*=lead]")
    else:
        lead_paragraph = float('NaN')

    lead_paragraph = lead_paragraph[0].text

    return lead_paragraph

In [ ]:
get_mis_description()

'Monthly company and individual insolvency statistics for England & Wales, and Northern Ireland, as well as company insolvency statistics for Scotland. Breathing Space scheme statistics and numbers of company moratoriums and restructuring plans are also included.'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()